<a href="https://colab.research.google.com/github/BRAHIMLOUARDI/movie-web-app/blob/main/Welcome_To_Colaboratory1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
# import string
# import numpy as np

# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Model,Sequential
# from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
# from tensorflow.keras.optimizers import Adam
# from keras.losses import sparse_categorical_crossentropy

# Path to translation filekk
path_to_data = 'fra_eng_sc.txt'

# Read file
translation_file = open(path_to_data,"r", encoding='utf-8') 
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('\t') for sentence in  raw_data]
# pairs=pairs[0:len(pairs)]



In [ ]:
x=0
for index,pair in enumerate(pairs):
  print(pair[1])
  x=index
  print(x)

print(pairs[58270])


In [2]:
!unzip  fra-eng.zip

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [4]:
def smart_find(haystack, needle):
    if haystack.startswith(needle+" "):
        return True
    if haystack.endswith(" "+needle):
        return True
    if haystack.find(" "+needle+" ") != -1:
        return True
    return False

In [7]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = "?" + "¡" + '¿'+"."+"!" ;
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

In [ ]:
from Book1 import words 
raw_data1=raw_data[0:60000]
raw_data2=[]
cout=0
for index,raw in enumerate(raw_data1):
  word_in_raw=False
  for word in words:
    clean_raw=clean_sentence(raw)
    if smart_find(clean_raw,word['English']) and not smart_find(clean_raw,word['French']):
      print(index)
      print('\n')
      print(word)
      print('\n')
      print(clean_raw)
      print('\n')
      print('---------')
      
      word_in_raw=True
  if not word_in_raw :
    cout=cout+1
    raw_data2.append(raw)




In [106]:
raw_data3=raw_data[0:len(raw_data)]

# for word in words:
#   z=word['English']+'\t'+word['French']
#   raw_data3.append(z)





In [108]:
for word in words:
  z=word['English']+'\t'+word['French']
  raw_data3.append(z)

In [92]:
# from Book1 import words 
# words2=words
# x=words1[1]['English']
# print(x.split('\/'))
for word in words1:
  word['English']=word['English'][0]
  word['French']=word['French'][0]
  word['Arabic']=word['Arabic'][0]


In [ ]:
for index, word in enumerate(words1):
   if 'Hardy-Weinberg principle'==word['English']:
       print(index)
       print(word)

In [109]:
with open('fra_eng_sc.txt', 'w') as f:
    for raw in raw_data3:
        f.write("%s\n" % raw)

In [71]:
print(len(words))

1445


In [8]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [36]:

english_sentences = [clean_sentence(pair[0]) for pair in pairs]
french_sentences = [clean_sentence(pair[1]) for pair in pairs]

# Tokenize words
fra_text_tokenized, fra_text_tokenizer = tokenize(french_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length french sentence: {}'.format(len(max(fra_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))


# Check language length
french_vocab = len(fra_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("french vocabulary is of {} unique words".format(french_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))

Maximum length french sentence: 14
Maximum length english sentence: 8
french vocabulary is of 15076 unique words
English vocabulary is of 7537 unique words


In [ ]:
print(french_sentences[50000])

n'est-ce pas agaçant


In [37]:
max_french_len = int(len(max(fra_text_tokenized,key=len)))
max_english_len = int(len(max(eng_text_tokenized,key=len)))

fra_pad_sentence = pad_sequences(fra_text_tokenized,15, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized,15, padding = "post")




fra_pad_sentence = fra_pad_sentence.reshape(*fra_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [38]:
input_sequence = Input(shape=(15,))
embedding = Embedding(input_dim=french_vocab, output_dim=128,)(input_sequence)

In [39]:
encoder = LSTM(64, return_sequences=False)(embedding)
r_vec = RepeatVector(15)(encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(english_vocab))(decoder)


In [40]:
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 15)]              0         
                                                                 
 embedding (Embedding)       (None, 15, 128)           1929728   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 repeat_vector (RepeatVector  (None, 15, 64)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 15, 64)            33024     
                                                                 
 time_distributed (TimeDistr  (None, 15, 7537)         489905    
 ibuted)                                                     

In [ ]:
BATCH_SIZE = 32
STEPS_PER_EPOCH = len(fra_pad_sentence) / BATCH_SIZE
SAVE_PERIOD = 10


from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("model/weights1.{epoch:02d}.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', save_freq=int(SAVE_PERIOD * STEPS_PER_EPOCH))

In [ ]:
print(int(SAVE_PERIOD * STEPS_PER_EPOCH))

20000


In [41]:
model_results = enc_dec_model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=32, epochs=100)


Epoch 1/100
1867/1867 [==============================] - 25s 9ms/step - loss: 1.7875 - accuracy: 0.7626
Epoch 2/100
1867/1867 [==============================] - 18s 9ms/step - loss: 1.5558 - accuracy: 0.7661
Epoch 3/100
1867/1867 [==============================] - 17s 9ms/step - loss: 1.5432 - accuracy: 0.7662
Epoch 4/100
1867/1867 [==============================] - 18s 9ms/step - loss: 1.4528 - accuracy: 0.7758
Epoch 5/100
1867/1867 [==============================] - 18s 10ms/step - loss: 1.3137 - accuracy: 0.7901
Epoch 6/100
1867/1867 [==============================] - 17s 9ms/step - loss: 1.1749 - accuracy: 0.8066
Epoch 7/100
1867/1867 [==============================] - 18s 9ms/step - loss: 1.0697 - accuracy: 0.8200
Epoch 8/100
1867/1867 [==============================] - 18s 9ms/step - loss: 0.9906 - accuracy: 0.8287
Epoch 9/100
1867/1867 [==============================] - 18s 9ms/step - loss: 0.9256 - accuracy: 0.8355
Epoch 10/100
1867/1867 [==============================] - 18s 9

In [42]:
enc_dec_model.save('my_model_v3.h5')



from google.colab import files
files.download('/content/my_model_v3.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# import tensorflow as tf
# new_enc_dec_model = tf.keras.models.load_model('my_model.h5')



In [ ]:
x=["logique"]
y=fra_text_tokenizer.texts_to_sequences(x)
z=pad_sequences(y, 15, padding = "post")

# print(y)
z=np.reshape(z,(1,15,1))

def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

index = 8000
# print("The english sentence is: {}".format(english_sentences[index]))
print("The french sentence is: {}".format(x[0]))
print('The predicted sentence is :')
print(logits_to_sentence(new_enc_dec_model.predict(z)[0], eng_text_tokenizer))






In [ ]:
 # Lower case the sentence
lower_case_sent = "N'est-ce pas agaçan¡t!.".lower()
print(lower_case_sent)
# Strip punctuation
string_punctuation = "?" + "¡" + '¿'+"."+"!" ;
print(string_punctuation)

clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
print(clean_sentence)
print(string.punctuation)

n'est-ce pas agaçan¡t!.
?¡¿.!
n'est-ce pas agaçant
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


exp

In [ ]:
#exp

input_sequence1 = Input(shape=(12,))
embedding1 = Embedding(input_dim=7188, output_dim=128,)(input_sequence1)

model1 = Model(inputs=input_sequence1 , outputs=embedding1, name="mnist_model")
model1.compile('rmsprop', 'mse')


input_data = np.array([[1,2,4,5,18,0,0,0,4,10,12,13]])

output_array = model1.predict(input_data)
print(output_array[0][0])


[ 0.02034706  0.02248744 -0.03376602 -0.01687739 -0.03728447 -0.00209639
 -0.00100971 -0.02335948  0.00595356  0.02687982  0.03717594  0.02074369
 -0.04843444 -0.00176772 -0.00239968 -0.04342381  0.02203074  0.00309337
  0.00114974  0.04493412 -0.00245256 -0.017037   -0.04806034 -0.02295033
  0.04644542 -0.02189548  0.04477992  0.03860844  0.0291361   0.03252422
  0.02577506  0.0369927  -0.01474409  0.02011487  0.04395882  0.03625769
  0.02296979 -0.00022303 -0.04299204 -0.00083683 -0.00481214  0.03876538
 -0.04163405 -0.00957751 -0.00837046 -0.03523744  0.03373507 -0.02530587
 -0.00866864  0.02286681  0.00869872 -0.01245948  0.04525365  0.01941318
  0.04451017 -0.04003798 -0.02288283 -0.00895053 -0.03188037  0.02156677
  0.01939699 -0.04546351  0.03530386  0.02994759  0.04775039  0.03900445
  0.01326055 -0.00981707 -0.04240481 -0.021184   -0.04985572 -0.02521235
 -0.01867831  0.0159093   0.0180388  -0.02206373  0.04946562  0.04724665
  0.03032454 -0.02553148 -0.01261898  0.02932639 -0

In [ ]:
# for layer in model1.layers: print(layer.get_config(), layer.get_weights())
#print(model1.layers[1].get_weights()[0].shape)

(7188, 128)


In [ ]:
#exp
lstm = LSTM(64, return_sequences=False)(embedding1)

lstmrepeatvector = RepeatVector(6)(lstm)
lstmdecoder = LSTM(64, return_sequences=True, dropout=0.2)(lstmrepeatvector)
lstmlogits = TimeDistributed(Dense(3733))(lstmdecoder)
modellstm = Model(inputs=input_sequence1 , outputs=lstmlogits, name="mnist_model")

In [ ]:
#exp
modellstm.compile('rmsprop', 'mse')


In [ ]:
modellstm.save('my_model_v21.h5')
!pip install tensorflowjs
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(modellstm, 'model')
!zip -r model.zip model

In [ ]:
#EXP
output_array = modellstm.predict(input_data)
reshape_out=np.reshape(output_array[0][0],(3733,-1))
print(reshape_out.shape)


(3733, 1)


In [ ]:
# list_of_lists = [[1,2,3],[4,5,6],[7,8,9]]

# import csv

# with open("out.csv", "w") as f:
#     wr = csv.writer(f)
   
#     wr.writerows(exppairs)

exppairs=pairs[0:10]
file = open("sample.py", "w+")

# Saving the array in a text file
for pair in exppairs :
   file.write(str(pair))
file.close()


In [ ]:
# modelexp = tf.sequential();
# modelexp.add(tf.layers.dense({ units: 1, inputShape: [1] }));

In [ ]:
import tensorflow as tf
import keras as keras
modelexp = Sequential()
# modelexp.add(Input(shape=(1,)))
modelexp.add(Dense(1,input_dim=1))

modelexp.compile(optimizer='sgd', loss=tf.keras.losses.MeanSquaredError( name="mean_squared_error"),metrics=['accuracy'])
xs = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]);
xs=np.reshape(xs,(12,1))
ys = np.array([1, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]);
ys=np.reshape(ys,(12,1))
modelexp.fit(xs, ys,epochs=3000, batch_size=12 )
x1=np.array([40])
x1=np.reshape(x1,(1,1))
print(x1.shape);
print(modelexp.predict(x1))

Streaming output truncated to the last 5000 lines.
Epoch 502/3000
1/1 [==============================] - 0s 6ms/step - loss: 0.0592 - accuracy: 0.0833
Epoch 503/3000
1/1 [==============================] - 0s 13ms/step - loss: 0.0592 - accuracy: 0.0833
Epoch 504/3000
1/1 [==============================] - 0s 6ms/step - loss: 0.0592 - accuracy: 0.0833
Epoch 505/3000
1/1 [==============================] - 0s 9ms/step - loss: 0.0592 - accuracy: 0.0833
Epoch 506/3000
1/1 [==============================] - 0s 7ms/step - loss: 0.0592 - accuracy: 0.0833
Epoch 507/3000
1/1 [==============================] - 0s 9ms/step - loss: 0.0592 - accuracy: 0.0833
Epoch 508/3000
1/1 [==============================] - 0s 10ms/step - loss: 0.0591 - accuracy: 0.0833
Epoch 509/3000
1/1 [==============================] - 0s 11ms/step - loss: 0.0591 - accuracy: 0.0833
Epoch 510/3000
1/1 [==============================] - 0s 10ms/step - loss: 0.0591 - accuracy: 0.0833
Epoch 511/3000
1/1 [=========================

In [ ]:
modelexp.save('my_model_v21.h5')
!pip install tensorflowjs
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(modelexp, 'model1')
!zip -r model.zip model

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
!pip install -q keras

In [ ]:
import tensorflow as tf
import keras as keras
modelexp = Sequential()
# modelexp.add(Input(shape=(1,)))
modelexp.add(Dense(1,input_dim=1))

modelexp.compile(optimizer='sgd', loss=tf.keras.losses.MeanSquaredError( name="mean_squared_error"),metrics=['accuracy'])
xs = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]);
xs=np.reshape(xs,(12,1))
ys = np.array([1, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]);
ys=np.reshape(ys,(12,1))
modelexp.fit(xs, ys,epochs=300, batch_size=1 )
x1=np.array([40])
x1=np.reshape(x1,(1,1))
print(x1.shape);
print(modelexp.predict(x1))

Epoch 1/300
12/12 [==============================] - 2s 3ms/step - loss: 22.6488 - accuracy: 0.0833 
Epoch 2/300
12/12 [==============================] - 0s 2ms/step - loss: 0.2788 - accuracy: 0.0833
Epoch 3/300
12/12 [==============================] - 0s 4ms/step - loss: 0.2171 - accuracy: 0.0833
Epoch 4/300
12/12 [==============================] - 0s 3ms/step - loss: 0.2407 - accuracy: 0.0833
Epoch 5/300
12/12 [==============================] - 0s 4ms/step - loss: 0.2351 - accuracy: 0.0833
Epoch 6/300
12/12 [==============================] - 0s 4ms/step - loss: 0.2086 - accuracy: 0.0833
Epoch 7/300
12/12 [==============================] - 0s 4ms/step - loss: 0.1788 - accuracy: 0.0833
Epoch 8/300
12/12 [==============================] - 0s 3ms/step - loss: 0.3320 - accuracy: 0.0833
Epoch 9/300
12/12 [==============================] - 0s 4ms/step - loss: 0.2088 - accuracy: 0.0833    
Epoch 10/300
12/12 [==============================] - 0s 8ms/step - loss: 0.1592 - accuracy: 0.0833
Epo